# Setup camera calibration plane

In [ ]:
from time import sleep
import sys

sys.path.append('/home/universal/Downloads/dev.sk_robot_rgbd_data/src')
from skrgbd.devices.robot.robot_on_plane import RobotOnPlane

In [ ]:
robot = RobotOnPlane()

1. Position the robot at bottom right of the calibration plane.

In [ ]:
bottom = 0
right = 1
robot.move_to((right, bottom), .1);

2. Position the calibration board
    * at the working distance of the cameras,
    * so that the tag is at the top boundary of all the images, and the left boundary of all the images,
    * so that the board is parallel to the image planes.

Make sure that the white area around the tag is fully visible on the images.

For monitoring use
* `tcam-capture` for TIS,
* `Protonect` for Kinect,
* `realsense-viewer` for RealSense,
* camera app on the screen of the phones.

3. Find the horizontal position of the robot so that the tag is at the right boundary of all the images.

In [ ]:
left = 0
robot.move_to((left, bottom), .1);

4. Find the vertical position of the robot so that the tag is at the bottom boundary of all the images.

In [ ]:
top = 1
robot.move_to((left, top), .1);

In [ ]:
robot.move_to((.4, top), .1);

In [ ]:
robot.move_to((right, top), .1);

5. Check the corners of all images.

In [ ]:
while True:
    robot.move_to((left, top), .1)
    robot.move_to((right, top), .1)
    robot.move_to((right, bottom), .1)
    robot.move_to((left, bottom), .1)

## Find tag crop positions

In [ ]:
from time import sleep
import sys

sys.stderr = sys.__stderr__  # redirect stderr back to the shell

from pupil_apriltags import Detector
import numpy as np
import matplotlib.pyplot as plt

sys.path.append('/home/universal/Downloads/dev.sk_robot_rgbd_data/src')
from skrgbd.calibration.calibration_helper.trajectories.camera_plane import CameraCalibrationPlane, make_im_slice
from skrgbd.devices.rig import Rig
from skrgbd.devices.robot.robot_on_plane import RobotOnPlane

In [ ]:
robot = RobotOnPlane()
trajectory = CameraCalibrationPlane(robot)
trajectory.move_zero(.1)

In [ ]:
rig = Rig()
rig.init_cameras().join()
rig.start_cameras().join()

In [ ]:
detector = Detector(
    families='tag36h11',
    nthreads=1,
    quad_decimate=1.0,
    quad_sigma=0.0,
    refine_edges=1,
    decode_sharpening=0.25,
    debug=0
)

In [ ]:
# image = rig.realsense.snap_frame(compressed=True)['ir_right']
# image = rig.tis_right.snap_frame()['image']
# image = rig.kinect.snap_frame()['ir']
# image = np.clip(image * 255, 0, 255).astype(np.uint8)
image = rig.phone_right.get_photo()

if image.ndim > 2:
    image = image[..., 1]  # keep only green channel since tag detector only works with single-channel images
plt.imshow(image, cmap='gray')

In [ ]:
detection = detector.detect(image)[0]

x, y = detection.center
size = (detection.corners[0, 0] - detection.center[0]) * 2

size = size * 1.3
size = round(size)
x = round(x)
y = round(y)

print(f'({y}, {x}), {size}')
plt.imshow(image[make_im_slice([y, x], size)], cmap='gray')

## Check tag crop positions

In [ ]:
from time import sleep
import sys

sys.stderr = sys.__stderr__  # redirect stderr back to the shell

sys.path.append('/home/universal/Downloads/dev.sk_robot_rgbd_data/src')
from skrgbd.calibration.calibration_helper.trajectories.camera_plane import CameraCalibrationPlane
from skrgbd.devices.rig import Rig

In [ ]:
rig = Rig()
rig.init_cameras().join()
rig.start_cameras().join()

In [ ]:
trajectory = CameraCalibrationPlane(None)
trajectory.stream_tag(rig.realsense, rig.tis_left, rig.tis_right, rig.kinect, rig.phone_left, rig.phone_right)

In [ ]:
trajectory.stop_tag_streaming()